In [1]:
from fastT5 import export_and_get_onnx_model, OnnxT5, get_onnx_runtime_sessions, get_onnx_model
from transformers import AutoTokenizer

In [5]:
base_model_path = '/home/falk/summarization/outputs/t5-large-paraphrased_twitter/t5-large-paraphrased_twitter(1)/best_model'
# model_or_model_path = '/home/falk/paraphrasing/outputs/t5-large/t5-large-paraphrasing/best_model'
model = export_and_get_onnx_model(base_model_path, input_sequence_length=1000)
# get_onnx_runtime_sessions("models")
# model = OnnxT5(model_name)

Exporting to onnx... |################################| 3/3
Quantizing... |################################| 3/3


Setting up onnx model...
Done!


In [2]:
t5_large_paths = "models/best_model-encoder.onnx", "models/best_model-decoder.onnx", "models/best_model-init-decoder.onnx"
model = OnnxT5("t5-large", get_onnx_runtime_sessions(t5_large_paths, default=False))
# model = model.to('cuda')

2022-01-17 14:07:00.061248270 [W:onnxruntime:, inference_session.cc:506 RegisterExecutionProvider] Parallel execution mode does not support the CUDA Execution Provider. So making the execution mode sequential for this session since it uses the CUDA Execution Provider.
2022-01-17 14:07:01.954480773 [W:onnxruntime:, inference_session.cc:506 RegisterExecutionProvider] Parallel execution mode does not support the CUDA Execution Provider. So making the execution mode sequential for this session since it uses the CUDA Execution Provider.
2022-01-17 14:07:02.952320974 [W:onnxruntime:, inference_session.cc:506 RegisterExecutionProvider] Parallel execution mode does not support the CUDA Execution Provider. So making the execution mode sequential for this session since it uses the CUDA Execution Provider.


In [4]:
model_name = "t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [6]:
from datasets import load_from_disk
dataset = load_from_disk("/home/falk/summarization/datasets/paraphrased_twitter")
input_prepend = "Summarize: "
inputs = dataset['test'][2]["article"]
text = inputs
token = tokenizer(text, padding="max_length", max_length=1024, return_tensors='pt')
# token = token.to('cuda')

In [7]:
tokens = model.generate(input_ids=token['input_ids'],
               attention_mask=token['attention_mask'],
               num_beams=1,
               max_length=200)

In [8]:
output = tokenizer.batch_decode(tokens, skip_special_tokens=True)
print(output)

["The winter storm brings wind, rain, snow to the US west A powerful storm packed heavy rain, snow and wind pounded parts of California and western Nevada early Wednesday. The Washington Post reported that it could be the largest snowfall the area has seen in two years. Geider's suggestion to those expecting to travel is to take extra time and be prepared. The storm will bring the heaviest snow to eastern Iowa, southern Wisconsin, northern Illinois, and parts of Indiana and Ohio, The Weather Channel reported. ABC reports that Chicago may see up to ten inches of snow. The heavy precipitation was a result of an atmospheric river, which are thousands of miles long ribbons of water vapor, that strengthen rain and snow in the West Coast."]


In [26]:
shape = token['attention_mask'].shape
list(shape)


[1, 1024]

In [7]:
import torch
x = torch.ones(10)
# x = x.to('cuda')
print(x.device)
print(x.device.index)

cpu
None


In [8]:
import onnxruntime
import numpy as np
X = np.ones(10)
x_ortvalue = onnxruntime.OrtValue.ortvalue_from_numpy(X, 'cuda', 0)
X.dtype
np.longlong

dtype('float64')

In [8]:
x_ortvalue.shape()

[10]

In [6]:
token

{'input_ids': tensor([[3111, 4000, 1575,  ...,    0,    0,    0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}

In [12]:

flat_past_key_values = [10, 20, 30]
past_key_values = [
    (f"pkv_{i}", pkv) for i, pkv in enumerate(flat_past_key_values)
]

past_key_values.append(list((f"pkv_{i}", pkv) for i, pkv in enumerate(flat_past_key_values))[0])
past_key_values

[('pkv_0', 10), ('pkv_1', 20), ('pkv_2', 30), ('pkv_0', 10)]